In [1]:
#!rm -r /home/jovyan/.local/share/jupyter/kernels/nilmtk
!mamba install tensorflow-gpu==2.10 -y -q
# !mamba install tensorflow-gpu==2.11.0 -y -q
!pip uninstall -y -q nilmtk nilm_metadata
#!pip install -q pandas networkx tables scikit-learn hmmlearn pyyaml matplotlib==3.1.3 xgboost pyts
# Trick to install NILM regardless of its dependencies
#!pip install --no-cache -U git+https://github.com/nilmtk/nilm_metadata.git
#!pip install --no-cache -U git+https://github.com/nilmtk/nilmtk.git@0.4.3
!python3 -m pip install --no-deps git+https://github.com/nilmtk/nilmtk@master
!python3 -m pip install --no-deps git+https://github.com/nilmtk/nilm_metadata@master

  Cloning https://github.com/nilmtk/nilmtk (to revision master) to /tmp/pip-req-build-xiwb7185
  Running command git clone --filter=blob:none --quiet https://github.com/nilmtk/nilmtk /tmp/pip-req-build-xiwb7185
  Resolved https://github.com/nilmtk/nilmtk to commit b2c514479cef478cab872cb635056da08d5352a1
  Preparing metadata (setup.py) ... done
  Created wheel for nilmtk: filename=nilmtk-0.4.0.dev1+git.b2c5144-py3-none-any.whl size=279177 sha256=5f5d02e7bdd77a2e1d0f26a040de048c91fc4142b29ead38380e141eeea8ef8b
  Stored in directory: /tmp/pip-ephem-wheel-cache-tfgqw5ax/wheels/05/a4/8e/b767c3d1714f61fd30ec991c25780be8cc04c474da8205aeee
Successfully built nilmtk
  Cloning https://github.com/nilmtk/nilm_metadata (to revision master) to /tmp/pip-req-build-6z4ygo0e
  Running command git clone --filter=blob:none --quiet https://github.com/nilmtk/nilm_metadata /tmp/pip-req-build-6z4ygo0e
  Resolved https://github.com/nilmtk/nilm_metadata to commit 7ed4bab9062d04cb35c6b6000b451715dc5ab4af
  Prep

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# from nilmtk import DataSet
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict
from joblib import Memory
import pickle
from pprint import pprint

#from nilmtk import DataSet
import multiprocessing as mp

from typing import Dict

!pip install -q scikit-learn-intelex
#from sklearnex import patch_sklearn
#patch_sklearn()

from sklearn import pipeline, metrics, linear_model, model_selection, multioutput, tree, ensemble, neural_network
!pip install xgboost -q
import xgboost as xgb

import numpy as np
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score
from sklearn import datasets, linear_model
from sklearn.model_selection import cross_val_predict, train_test_split
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import LearningRateScheduler

import numpy as np
#import graphviz
import keras
from sklearn.utils import class_weight
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator
#from ann_visualizer.visualize import ann_viz
from matplotlib import pyplot as plt 
from sklearn.metrics import multilabel_confusion_matrix, classification_report, confusion_matrix
# GPU goes brrrrrrrrrrrrrrrrrrrr
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

!pip3 install tqdm -q
from tqdm.notebook import tqdm
import NUK

# import garbage collector
import gc
import time


physical_devices = tf.config.list_physical_devices('GPU')

if physical_devices:
    print(physical_devices)
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
else:
    print("No GPU found, model will train on CPU.")


2023-10-24 06:15:39.143411: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA A100 80GB PCIe MIG 3g.40gb, compute capability 8.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Preprocessing

# Models

In [3]:
# Parameters

epochs =400
window_size = 2688
batch_size = 128
NmDevices = 64
# table_of_options = [1,2,3,4]
k = 0.125
# Define a learning rate scheduler function
def scheduler(epoch, lr):
    return lr
    if epoch == 0 or epoch == 1:
        return lr
    if epoch == 35:
        return lr *0.5
    else:
        return lr

RANDOM_SEED = 42
test_size = 0.2
lambda_l2=0
function = "GRU"

In [4]:
data= pd.read_pickle("../../Energy_graph/data/training_data/processed/X_Y_wsize2688_upper32_gap3600_numD64.pkl")
# data = pd.read_pickle("../../Energy_graph/data/training_data/synthetic/X_Y_wsize2688_numW_50000_upper32_gap3600_numD64_ideal.pkl") 
data_syn_ideal = pd.read_pickle("../../Energy_graph/data/training_data/synthetic/X_Y_wsize2688_numW_50000_upper32_gap3600_numD64_ideal.pkl") 
data_syn_unmetered = pd.read_pickle("../../Energy_graph/data/training_data/synthetic/X_Y_wsize2688_numW_50000_upper32_gap3600_numD64_unmetered.pkl")

labels = pd.read_pickle("../../Energy_graph/data/labels_new.pkl")


# Separate the tuples into X and y
X_syn_ideal = np.array([i[0] for i in data_syn_ideal])
y_syn_ideal = np.array([i[1] for i in data_syn_ideal])

X_syn_unmetered = np.array([i[0] for i in data_syn_unmetered])
y_syn_unmetered = np.array([i[1] for i in data_syn_unmetered])

X = np.array([i[0] for i in data])
y = np.array([i[1] for i in data])


X = np.concatenate((X, X_syn_ideal,X_syn_unmetered), axis=0)
y = np.concatenate((y, y_syn_ideal, y_syn_unmetered), axis=0)


y = y.astype(int)

# X_test = np.array([i[0] for i in data_test])
# y_test = np.array([i[1] for i in data_test])
def normalize(X):
    max_value = 0

    for x in X:
        v = np.max(x)
        if v > max_value:
            max_value = v
    return X / max_value

    




In [5]:
len(labels)

64

## InceptionTime

### InceptionTime class

In [6]:
class_weighs_pre = NUK.class_weights_tool(y)

class Classifier_INCEPTION:

    def __init__(self, output_directory, input_shape, nb_classes, iteration, verbose=False, build=True, batch_size=64,
                 nb_filters=32, use_residual=True, use_bottleneck=True, depth=6, kernel_size=41, nb_epochs=1500, lr=0.001):

        self.output_directory = output_directory

        self.nb_filters = nb_filters
        self.use_residual = use_residual
        self.use_bottleneck = use_bottleneck
        self.depth = depth
        self.kernel_size = kernel_size - 1
        self.callbacks = None
        self.batch_size = batch_size
        self.bottleneck_size = 32
        self.nb_epochs = nb_epochs
        self.lr = lr
        self.iteration = iteration

        if build == True:
            self.model = self.build_model(input_shape, nb_classes)
            # if (verbose == True):
                # self.model.summary()
            self.verbose = verbose
            self.model.save_weights(self.output_directory + 'model_init.hdf5')

    def _inception_module(self, input_tensor, stride=1, activation='linear'):

        if self.use_bottleneck and int(input_tensor.shape[-1]) > 1:
            input_inception = keras.layers.Conv1D(filters=self.bottleneck_size, kernel_size=1,
                                                  padding='same', activation=activation, use_bias=False)(input_tensor)
        else:
            input_inception = input_tensor

        # kernel_size_s = [3, 5, 8, 11, 17]
        kernel_size_s = [self.kernel_size // (2 ** i) for i in range(3)]

        conv_list = []

        for i in range(len(kernel_size_s)):
            conv_list.append(keras.layers.Conv1D(filters=self.nb_filters, kernel_size=kernel_size_s[i],
                                                 strides=stride, padding='same', activation=activation, use_bias=False)(
                input_inception))

        max_pool_1 = keras.layers.MaxPool1D(pool_size=3, strides=stride, padding='same')(input_tensor)

        conv_6 = keras.layers.Conv1D(filters=self.nb_filters, kernel_size=1,
                                     padding='same', activation=activation, use_bias=False)(max_pool_1)

        conv_list.append(conv_6)

        x = keras.layers.Concatenate(axis=2)(conv_list)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Activation(activation='relu')(x)
        return x

    def _shortcut_layer(self, input_tensor, out_tensor):
        shortcut_y = keras.layers.Conv1D(filters=int(out_tensor.shape[-1]), kernel_size=1,
                                         padding='same', use_bias=False)(input_tensor)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        x = keras.layers.Add()([shortcut_y, out_tensor])
        x = keras.layers.Activation('relu')(x)
        return x

    def build_model(self, input_shape, nb_classes):
        input_layer = keras.layers.Input(input_shape)

        x = input_layer
        input_res = input_layer

        for d in range(self.depth):

            x = self._inception_module(x)

            if self.use_residual and d % 3 == 2:
                x = self._shortcut_layer(input_res, x)
                input_res = x

        gap_layer = keras.layers.GlobalAveragePooling1D()(x)

        output_layer = keras.layers.Dense(nb_classes, activation='sigmoid')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=self.lr),
                      metrics=['accuracy', NUK.F1Score, NUK.WeightedF1Score(class_weighs_pre)])

        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.25, patience=7, min_lr=0.0001)
        early_stopping = keras.callbacks.EarlyStopping(patience=9, restore_best_weights=True, monitor='loss')

        file_path = self.output_directory + f'best_model_{self.iteration}.hdf5'

        model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
                                                           save_best_only=True)
        # lr_scheduler = LearningRateScheduler(scheduler)
        self.callbacks = [reduce_lr, model_checkpoint, early_stopping]

        return model

    def fit(self, x_train, y_train):
       
        # x_val and y_val are only used to monitor the test loss and NOT for training

        if self.batch_size is None:
            mini_batch_size = int(min(x_train.shape[0] / 10, 16))
        else:
            mini_batch_size = self.batch_size

        self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=self.nb_epochs,
                                  verbose=self.verbose, callbacks=self.callbacks, validation_split=0.1)


        self.model.save(self.output_directory + f'last_model_{self.iteration}.hdf5')

        keras.backend.clear_session()

        return None

    def predict(self, x_test):
        model_path = self.output_directory + f'last_model_{self.iteration}.hdf5'
        model = keras.models.load_model(model_path, custom_objects={'F1Score': NUK.F1Score, "WeightedF1": NUK.WeightedF1Score(class_weighs_pre)})
        y_pred = model.predict(x_test, batch_size=self.batch_size)
    #     # if return_df_metrics:
    #     #     y_pred = np.argmax(y_pred, axis=1)
    #     #     df_metrics = calculate_metrics(y_true, y_pred, 0.0)
    #     #     return df_metrics
    #     # else:
    #     #     test_duration = time.time() - start_time
    #     #     save_test_duration(self.output_directory + 'test_duration.csv', test_duration)
    #     #     return y_pred
        return y_pred


### Single model

In [ ]:

kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
evaluation_results = []
predictions = []
class_weights_all = NUK.class_weights_tool(y)
itr = 0
for train_index, test_index in tqdm(kf.split(X)):
    # print("Train indices:", train_index, "Test indices:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train = normalize(X_train)
    X_test = normalize(X_test)
    # model = NUK.PC0_reg(NmDevices, window_size, function, 128, k, lambda_l2=lambda_l2)
    print(batch_size, epochs)
    model = Classifier_INCEPTION(output_directory="./models/", input_shape=(window_size, 1), nb_classes=78, verbose=True, build=True, batch_size=batch_size, nb_epochs=epochs, lr=0.001, depth=6)
    model.fit(X_train, y_train)
    # model.model.predict(X_test, batch_size=batch_size)

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_tf = (y_pred > 0.5)

    predictions.append((y_pred_tf, y_test, y_pred))
    t = (y_pred_tf, y_test, y_pred)
    with open(f"./predictions/test_inception_pickle_{itr}.pkl", 'wb') as f:
        pickle.dump(t, f)

    itr += 1
    report = metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)
    print(report["weighted avg"])
    # report_DF = NUK.ClassificationReportToDF(report, labels)  
    # report_DF.rename(columns={report_DF.columns[0]: "device"}, inplace=True)
    evaluation_results.append(report)
    gc.collect()
    tf.keras.backend.clear_session()

for i in range(len(evaluation_results)):
    if i == 0:
        df = pd.DataFrame(evaluation_results[i]).T
    else:
        df = df + pd.DataFrame(evaluation_results[i]).T


df = df / 5
import os
# check if directory exsits
if not os.path.exists(f"../../Energy_graph/data/results/{window_size}"):
    os.makedirs(f"../../Energy_graph/data/results/{window_size}")
if lambda_l2 == 0:
    df.to_csv(f"../../Energy_graph/data/results/{window_size}/Inception_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn.csv")
else:
    df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn_reg{lambda_l2}.csv")



    


### Ensemble

#### k-fold

In [31]:
num_splits = 3
kf = KFold(n_splits=num_splits, shuffle=True, random_state=RANDOM_SEED)
evaluation_results = []
predictions = []
class_weights_all = NUK.class_weights_tool(y)
for fold,(train_index, test_index) in tqdm(enumerate(kf.split(X))):
    # print("Train indices:", train_index, "Test indices:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # normalization
    X_train = normalize(X_train)
    X_test = normalize(X_test)
    model_predictions = []

    print(batch_size, epochs)
    for i in range(5):

        model = Classifier_INCEPTION(output_directory=f"./models/{fold}/", input_shape=(window_size, 1), nb_classes=78, verbose=True, build=True, batch_size=batch_size, nb_epochs=epochs, lr=0.001, depth=8, iteration=i)
        model.fit(X_train, y_train)

        # Evaluate the model
        y_pred = model.predict(X_test)
        model_predictions.append(y_pred)

    y_pred = np.mean(model_predictions, axis=0)
    y_pred_tf = (y_pred > 0.5)

    predictions.append((y_pred_tf, y_test, y_pred))
    t = (y_pred_tf, y_test, y_pred)
    with open(f"./predictions/test_inception_pickle_{fold}.pkl", 'wb') as f:
        pickle.dump(t, f)

    report = metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)
    print(report["weighted avg"])
    
    evaluation_results.append(report)
    gc.collect()
    tf.keras.backend.clear_session()

for i in range(len(evaluation_results)):
    if i == 0:
        df = pd.DataFrame(evaluation_results[i]).T
    else:
        df = df + pd.DataFrame(evaluation_results[i]).T


df = df / num_splits
import os
# check if directory exsits
if not os.path.exists(f"../../Energy_graph/data/results/{window_size}"):
    os.makedirs(f"../../Energy_graph/data/results/{window_size}")
if lambda_l2 == 0:
    df.to_csv(f"../../Energy_graph/data/results/{window_size}/NNE_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn_3fold.csv")
else:
    df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn_reg{lambda_l2}.csv")



    


KeyboardInterrupt: 

#### 80/20 split

In [8]:


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
X_train = normalize(X_train)
X_test = normalize(X_test)

evaluation_results = []
predictions = []
class_weights_all = NUK.class_weights_tool(y)
model_predictions = []

print(batch_size, epochs)
for i in range(5):

    model = Classifier_INCEPTION(output_directory=f"./models/test/", input_shape=(window_size, 1), nb_classes=NmDevices, verbose=True, build=True, batch_size=batch_size, nb_epochs=epochs, lr=0.001, depth=10, iteration=i)
    # model = keras.models.load_model(f"./models/test/last_model_{i}.hdf5", custom_objects={'F1Score': NUK.F1Score, "WeightedF1": NUK.WeightedF1Score(class_weighs_pre)})
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    model_predictions.append(y_pred)

y_pred = np.mean(model_predictions, axis=0)
y_pred_tf = (y_pred > 0.3)

predictions.append((y_pred_tf, y_test, y_pred))
t = (y_pred_tf, y_test, y_pred)
with open(f"./predictions/test_inception_pickle_d10.pkl", 'wb') as f:
    pickle.dump(t, f)

report = metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)
print(report["weighted avg"])

evaluation_results.append(report)
gc.collect()
tf.keras.backend.clear_session()

for i in range(len(evaluation_results)):
    if i == 0:
        df = pd.DataFrame(evaluation_results[i]).T
    else:
        df = df + pd.DataFrame(evaluation_results[i]).T

import os
# check if directory exsits
if not os.path.exists(f"../../Energy_graph/data/results/{window_size}"):
    os.makedirs(f"../../Energy_graph/data/results/{window_size}")
if lambda_l2 == 0:
    df.to_csv(f"../../Energy_graph/data/results/{window_size}/NNE_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn_mixed_t0.3_d10.csv")
else:
    df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn_reg{lambda_l2}.csv")



    


128 400
Epoch 1/400
827/827 [==============================] - 275s 326ms/step - loss: 0.2979 - accuracy: 0.1070 - F1Score: 0.0418 - WeightedF1: 0.0664 - val_loss: 0.2891 - val_accuracy: 0.1222 - val_F1Score: 0.0669 - val_WeightedF1: 0.0858 - lr: 0.0010
Epoch 2/400
827/827 [==============================] - 267s 323ms/step - loss: 0.2737 - accuracy: 0.1247 - F1Score: 0.0961 - WeightedF1: 0.1340 - val_loss: 0.2749 - val_accuracy: 0.1173 - val_F1Score: 0.1157 - val_WeightedF1: 0.1485 - lr: 0.0010
Epoch 3/400
827/827 [==============================] - 266s 322ms/step - loss: 0.2627 - accuracy: 0.1320 - F1Score: 0.1397 - WeightedF1: 0.1811 - val_loss: 0.2764 - val_accuracy: 0.0873 - val_F1Score: 0.1345 - val_WeightedF1: 0.1750 - lr: 0.0010
Epoch 4/400
827/827 [==============================] - 267s 323ms/step - loss: 0.2535 - accuracy: 0.1387 - F1Score: 0.1786 - WeightedF1: 0.2208 - val_loss: 0.3570 - val_accuracy: 0.0482 - val_F1Score: 0.1432 - val_WeightedF1: 0.1639 - lr: 0.0010
Epoch 5/

## ResNet

In [20]:
def build_resnet(input_shape, n_feature_maps, nb_classes):
    print ('build conv_x')
    x = keras.layers.Input(shape=(input_shape))
    conv_x = keras.layers.BatchNormalization()(x)
    conv_x = keras.layers.Conv1D(n_feature_maps, 8, 1, padding='same')(conv_x)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv1D(n_feature_maps, 5, 1, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv1D(n_feature_maps, 3, 1, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)
     
    is_expand_channels = not (input_shape[-1] == n_feature_maps)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv1D(n_feature_maps, 1, 1,padding='same')(x)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x)
    print ('Merging skip connection')
    y = keras.layers.Add()([shortcut_y, conv_z])
    y = keras.layers.Activation('relu')(y)
     
    print ('build conv_x')
    x1 = y
    conv_x = keras.layers.Conv1D(n_feature_maps*2, 8, 1, padding='same')(x1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv1D(n_feature_maps*2, 5, 1, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv1D(n_feature_maps*2, 3, 1, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)
     
    is_expand_channels = not (input_shape[-1] == n_feature_maps*2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv1D(n_feature_maps*2, 1, 1,padding='same')(x1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x1)
    print ('Merging skip connection')
    y = keras.layers.Add()([shortcut_y, conv_z])
    y = keras.layers.Activation('relu')(y)
     
    print ('build conv_x')
    x1 = y
    conv_x = keras.layers.Conv1D(n_feature_maps*2, 8, 1, padding='same')(x1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv1D(n_feature_maps*2, 5, 1, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv1D(n_feature_maps*2, 3, 1, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)
    is_expand_channels = not (input_shape[-1] == n_feature_maps*2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv1D(n_feature_maps*2, 1, 1,padding='same')(x1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x1)
    print ('Merging skip connection')
    y = keras.layers.Add()([shortcut_y, conv_z])
    y = keras.layers.Activation('relu')(y)
     
    full = keras.layers.GlobalAveragePooling1D()(y)
    out = keras.layers.Dense(nb_classes, activation='sigmoid')(full)
    print ('        -- model was built.')
    # input = keras.layers.Input(shape=(input_shape))
    model = keras.models.Model(inputs=x, outputs=out, name="Resnet")

    return model

In [ ]:
num_splits = 5
kf = KFold(n_splits=num_splits, shuffle=True, random_state=RANDOM_SEED)
evaluation_results = []
predictions = []
class_weights_all = NUK.class_weights_tool(y)
treshold = 0.5
debug = 0
for train_index, test_index in tqdm(kf.split(X)):
    print("Train indices:", train_index, "Test indices:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train[:, 0] = normalize(X_train[:, 0])
    X_test[:,0] = normalize(X_test[:, 0])

    model = build_resnet((window_size, 1), 32, NmDevices)
    # model.build((len(y_train) + len(y_test), window_size, 1))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=[NUK.F1Score, NUK.WeightedF1Score(NUK.class_weights_tool(y_test))])
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.25, patience=3, min_lr=0.000001)
    early_stopping = keras.callbacks.EarlyStopping(patience=6, restore_best_weights=True, monitor='loss')

    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, callbacks=[reduce_lr,early_stopping])

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_tf = (y_pred > treshold)

    predictions.append((y_pred, y_pred_tf, y_test))



    report = metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)
    print(report["weighted avg"], "|||", report["samples avg"])

    # report_DF = NUK.ClassificationReportToDF(report, labels)  
    # report_DF.rename(columns={report_DF.columns[0]: "device"}, inplace=True)
    evaluation_results.append(report)
    gc.collect()
    tf.keras.backend.clear_session()

for i in range(len(evaluation_results)):
    if i == 0:
        df = pd.DataFrame(evaluation_results[i]).T
    else:
        df = df + pd.DataFrame(evaluation_results[i]).T
df = df / num_splits
import os
# check if directory exsits
if not os.path.exists(f"../../Energy_graph/data/results/{window_size}"):
    os.makedirs(f"../../Energy_graph/data/results/{window_size}")
if lambda_l2 == 0:
    df.to_csv(f"../../Energy_graph/data/results/{window_size}/ResNet_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn_mixed_t0.3_d10.csv")
else:
    df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn_reg{lambda_l2}.csv")




    



## PC0

In [ ]:

kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
evaluation_results = []
predictions = []
class_weights_all = NUK.class_weights_tool(y)
treshold = 0.5
debug = 0
for train_index, test_index in tqdm(kf.split(X)):
    print("Train indices:", train_index, "Test indices:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train[:, 0] = normalize(X_train[:, 0])
    X_test[:,0] = normalize(X_test[:, 0])

    # break
    # class_weights_all = NUK.class_weights_tool(y_train)
    # for k in class_weights_all:
    #     class_weights_all[k] += 1
    
    # print(class_weights_all)
    # break
    model = NUK.PC0_reg(NmDevices, window_size, function, 128, k, lambda_l2=lambda_l2)
    # model = NUK.VGG11_1D(NmDevices, window_size)
    model.build((len(y_train) + len(y_test), window_size, 1))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=[NUK.F1Score, NUK.WeightedF1Score(NUK.class_weights_tool(y_test))])
    # lr_scheduler = LearningRateScheduler(scheduler)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.25, patience=3, min_lr=0.000001)
    # callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1,patience=2, min_lr=0.0000002)
    # for class weights
    # model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, class_weight=class_weights_UKD, callbacks=[lr_scheduler])
    # class_weights = NUK.class_weights_tool(y_train)
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, callbacks=[reduce_lr])

    # Evaluate the model
    y_pred = model.predict(X_test)
    y_pred_tf = (y_pred > treshold)

    predictions.append((y_pred, y_pred_tf, y_test))

    # n_labels = y_test.shape[1]
    # co_occurrence_matrix = np.zeros((n_labels, n_labels))

    # for true, pred in zip(y_test, y_pred_tf):
    #     fn_labels = np.where((true == 1) & (pred == 0))[0]  # False negatives
    #     fp_labels = np.where((true == 0) & (pred == 1))[0]  # False positives

    #     for fn in fn_labels:
    #         for fp in fp_labels:
    #             co_occurrence_matrix[fn, fp] += 1



    report = metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)
    print(report["weighted avg"], "|||", report["samples avg"])

    # report_DF = NUK.ClassificationReportToDF(report, labels)  
    # report_DF.rename(columns={report_DF.columns[0]: "device"}, inplace=True)
    evaluation_results.append(report)
    gc.collect()
    tf.keras.backend.clear_session()



    


## TSMixer
https://github.com/google-research/google-research/tree/master/tsmixer/tsmixer_basic/models

In [ ]:
#TODO implement TSMixer for multilabel classification

## Random forest

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
jobs = 110
kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
evaluation_results = []

for train_index, test_index in tqdm(kf.split(X)):
    print("Train indices:", train_index, "Test indices:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Initialize the Random Forest classifier wrapped in a MultiOutputClassifier
    classifier = RandomForestClassifier(n_estimators=100, random_state=RANDOM_SEED, n_jobs=jobs)
    # classifier = MultiOutputClassifier(forest, n_jobs=-1)
    classifier.fit(X_train, y_train)

    # Predict the labels
    y_pred = classifier.predict(X_test)

    # Evaluate the model using classification report
    report = classification_report(y_test, y_pred, target_names=labels, zero_division=0, output_dict=True)

    evaluation_results.append(report)


for i in range(len(evaluation_results)):
    if i == 0:
        df = pd.DataFrame(evaluation_results[i]).T
    else:
        df = df + pd.DataFrame(evaluation_results[i]).T


df = df / 5
# df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_staticLR_LSTM_watts_mixed100k.csv")
df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_classes{NmDevices}_watts_RF_mixed50k.csv")
    


0it [00:00, ?it/s]

Train indices: [     1      2      3 ... 147053 147054 147055] Test indices: [     0      4     12 ... 147049 147050 147052]


In [ ]:
evaluation_results = []
model = NUK.PC0(NmDevices, window_size, 'GRU',128, k)
model.build((len(y), window_size, 1))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003), loss='binary_crossentropy', metrics=[NUK.F1Score, NUK.WeightedF1Score(NUK.class_weights_tool(y_test))])
lr_scheduler = LearningRateScheduler(scheduler)

model.fit(X, y, batch_size=batch_size, epochs=epochs, callbacks=[lr_scheduler])


y_pred = model.predict(X_test)
y_pred_tf = (y_pred > 0.5)
report = metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)

# report_DF = NUK.ClassificationReportToDF(report, labels)  
# report_DF.rename(columns={report_DF.columns[0]: "device"}, inplace=True)
df = pd.DataFrame(report)

## CatBoost

In [4]:
%pip install catboost
import catboost as cb


Note: you may need to restart the kernel to use updated packages.


In [15]:


kf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
evaluation_results = []
predictions = []
class_weights_all = NUK.class_weights_tool(y)
debug = 0
for train_index, test_index in tqdm(kf.split(X)):
    print("Train indices:", train_index, "Test indices:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    X_train = normalize(X_train)
    X_test = normalize(X_test)

    # break

    # Initialize CatBoost multilabel classifier
    model = cb.CatBoostClassifier(
        loss_function='MultiLogloss',
        verbose=500,
        random_seed=RANDOM_SEED,
        task_type="GPU",
        devices='0:1',
        # class_weights=class_weights_all,
        iterations=500,

        )  # Add other hyperparameters if needed

    # Train the model
    model.fit(X_train, y_train)

    # Get predictions
    y_pred = model.predict(X_test)

    # Convert predictions to boolean format similar to y_test for evaluation
    y_pred_tf = (y_pred == 1)
    print(y_pred)

    predictions.append((y_pred_tf, y_test))




    report = metrics.classification_report(y_test, y_pred_tf, target_names=labels, zero_division=0, output_dict=True)
    print(report["weighted avg"])
    break
    # report_DF = NUK.ClassificationReportToDF(report, labels)  
    # report_DF.rename(columns={report_DF.columns[0]: "device"}, inplace=True)
    evaluation_results.append(report)
    del model



    


0it [00:00, ?it/s]

Train indices: [    0     1     2 ... 49997 49998 49999] Test indices: [    4     6     7 ... 49988 49990 49992]
Learning rate set to 0.051213
0:	learn: 0.6478853	total: 545ms	remaining: 4m 32s
499:	learn: 0.2732736	total: 3m 57s	remaining: 0us
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
{'precision': 0.12109148155906953, 'recall': 0.021307596513075965, 'f1-score': 0.03282443454939719, 'support': 80300}


In [16]:
y_pred[2]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [57]:
gc.collect()
tf.keras.backend.clear_session()


## Evaluation

In [12]:
import os
for i in range(len(evaluation_results)):
    if i == 0:
        df = pd.DataFrame(evaluation_results[i]).T
    else:
        df = df + pd.DataFrame(evaluation_results[i]).T


df = df / 5
# check if directory exsits
if not os.path.exists(f"../../Energy_graph/data/results/{window_size}"):
    os.makedirs(f"../../Energy_graph/data/results/{window_size}")
if lambda_l2 == 0:
    df.to_csv(f"../../Energy_graph/data/results/{window_size}/PC0_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn_mix.csv")
else:
    df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_epochs{epochs}_batch_size{batch_size}_k{k}_classes{NmDevices}_{function}_watts_syn_reg{lambda_l2}.csv")
# df.to_csv(f"../../Energy_graph/data/results/w_size{window_size}_classes{NmDevices}_watts_RF_mixed50k.csv")
    
# evaluation_results+ evaluation_results[1].to_